In [1]:
from functools import wraps
import collections
import os
import time

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, Binarizer, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/Users/zhouzhirui/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
kaggle_path = '../input/'
mac_path = '/Users/zhouzhirui/data/Mercari_Price_Forcast/'
def handle_missing(dataset):
        dataset.category.fillna(value="missing", inplace=True)
        dataset.brand.fillna(value="missing", inplace=True)
        dataset.description.fillna(value="missing", inplace=True)
        dataset.loc[dataset.description == 'No description yet', 'description'] = 'missing'
        return dataset
    
def upper2lower(dataset):
    for (col, dtype) in dataset.dtypes.iteritems():
        if dtype == 'object':
            dataset[col] = dataset[col].str.lower()
    return dataset

def load_data(path):
    print('load data ..')
    train = pd.read_table(path+'train.tsv')
    test = pd.read_table(path+'test.tsv')
    merge = pd.concat([train, test], axis=0).reset_index(drop=True)
    merge.rename(columns={
        'brand_name':'brand',
        'category_name':'category',
        'item_condition_id':'condition',
        'item_description':'description'
    }, inplace=True)
    print('handle missing ..')
    merge = handle_missing(merge)
    print('upper2lower ..')
    merge = upper2lower(merge)
    train = merge[merge.train_id.notnull()].reset_index(drop=True)
    test = merge[merge.train_id.isnull()].reset_index(drop=True)
    return train, test

train, test = load_data(mac_path)

load data ..
handle missing ..
upper2lower ..


In [19]:
train.head()

,brand,category,condition,description,name,price,shipping,test_id,train_id
0,missing,men/tops/t-shirts,3,missing,mlb cincinnati reds t shirt size xl,10.0,1,NaN,0.0
1,razer,electronics/computers & tablets/components & p...,3,this keyboard is in great condition and works ...,razer blackwidow chroma keyboard,52.0,0,NaN,1.0
2,target,women/tops & blouses/blouse,1,adorable top with a hint of lace and a key hol...,ava-viv blouse,10.0,1,NaN,2.0
3,missing,home/home décor/home décor accents,1,new with tags. leather horses. retail for [rm]...,leather horse statues,35.0,1,NaN,3.0
4,missing,women/jewelry/necklaces,1,complete with certificate of authenticity,24k gold plated rose,44.0,0,NaN,4.0


In [21]:
train.groupby(train['description'].apply(lambda x:1 if '[rm]' in x else 0))['price'].mean()

description
0    26.226363
1    31.889807
Name: price, dtype: float64

In [18]:
train.category.value_counts()

women/athletic apparel/pants, tights, leggings                 60177
women/tops & blouses/t-shirts                                  46380
beauty/makeup/face                                             34335
beauty/makeup/lips                                             29910
electronics/video games & consoles/games                       26557
beauty/makeup/eyes                                             25215
electronics/cell phones & accessories/cases, covers & skins    24676
women/underwear/bras                                           21274
women/tops & blouses/tank, cami                                20284
women/tops & blouses/blouse                                    20284
women/dresses/above knee, mini                                 20082
women/jewelry/necklaces                                        19758
women/athletic apparel/shorts                                  19528
beauty/makeup/makeup palettes                                  19103
women/shoes/boots                 